In [1]:
import json
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import requests
import yaml
from box import ConfigBox
from sklearn.metrics import f1_score
from tqdm import tqdm


### Set web service URL

In [4]:
# local testing
API_URL = "http://0.0.0.0:8080" 
# testing deployed service
# API_URL = "https://<YOUR_APP_NAME>.fly.dev" 
# API_URL = "https://mlops.fly.dev" 
# API_URL = "postgresql://aida-mlops.flycast" 

API_URL

'http://0.0.0.0:8080'

### Load feature columns and data

In [6]:
def load_params(params_path):
    with open(params_path, "r") as f:
        params = yaml.safe_load(f)
        params = ConfigBox(params)
    return params


proj_path = Path(os.getcwd()).parent.absolute()
params = load_params(proj_path/'params.yaml')
feat_cols = params.features.cols
targ_col = params.target
feat_cols, targ_col

(BoxList(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']),
 'Exited')

In [7]:
df = pd.concat([pd.read_csv(proj_path/'data'/'raw'/f'Churn_Modelling_{country}.csv') for country in ['France', 'Spain']])
df.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
2,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
3,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
4,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0


### Test `/predict` endpoint by sending one sample request

In [11]:
my_obj = {
    "data": [
            {
      "CreditScore": 619,
      "Age": 42,
      "Tenure": 2,
      "Balance": 0,
      "NumOfProducts": 1,
      "HasCrCard": 1,
      "IsActiveMember": 1,
      "EstimatedSalary": 101348.88
            }
        ]
    }
x = requests.post(API_URL + '/predict', json = my_obj)
if x.ok:
    probs = x.json()
else:
    x.raise_for_status()

probs

[0.761036968147323]

### Create a functions for calling `/predict` endpoint

In [22]:
def get_prob(obj):
    x = requests.post(API_URL + '/predict', json = obj)
    if x.ok:
        probs = x.json()
    else:
        x.raise_for_status()
    return probs

get_prob(my_obj)

[0.761036968147323]

In [20]:
# def get_prob(obj):
#     resp = requests.post(API_URL + "/predict", json=obj)
#     return resp.json()["probs"]  # Return only the list of floats


### Check what would f1-scores look like if we send the data from the same geographies that the model was trained on 

In [23]:
import time


def send_sample_requests(df):
    f1_score_list = []
    for _ in tqdm(range(50)):
        df_sample = df.sample(n=60)
        y_true = df_sample[targ_col].values
        obj = {"data": df_sample[feat_cols].to_dict('records')}
        probs = get_prob(obj)
        
        y_pred = np.array([prob < 0.5 for prob in probs]).astype(int)

        f1 = f1_score(y_true, y_pred)
        f1_score_list.append(f1)
        time.sleep(1)
    return f1_score_list
        
f1_score_list = send_sample_requests(df)
f1_score_list

100%|██████████| 50/50 [00:52<00:00,  1.05s/it]


[0.5454545454545454,
 0.6666666666666665,
 0.631578947368421,
 0.7499999999999999,
 0.28571428571428575,
 0.782608695652174,
 0.4444444444444444,
 0.888888888888889,
 0.8,
 0.6666666666666666,
 0.5714285714285715,
 0.5333333333333333,
 0.6153846153846153,
 0.7000000000000001,
 0.6666666666666666,
 0.6,
 0.6153846153846153,
 0.5714285714285715,
 0.5,
 0.47058823529411764,
 0.6666666666666666,
 0.25,
 0.3636363636363636,
 0.7368421052631579,
 0.7142857142857143,
 0.4615384615384615,
 0.6666666666666666,
 0.7368421052631579,
 0.6,
 0.8571428571428571,
 0.4,
 0.4444444444444445,
 0.47058823529411764,
 0.5,
 0.4285714285714285,
 0.5,
 0.5333333333333333,
 0.6,
 0.3809523809523809,
 0.2222222222222222,
 0.7499999999999999,
 0.631578947368421,
 0.2,
 0.7142857142857143,
 0.6,
 0.6666666666666666,
 0.7272727272727272,
 0.25,
 0.888888888888889,
 0.7058823529411764]

### What would f1-scores look like for input data from new geography?

In [24]:
df_germany =  pd.read_csv(proj_path/'data'/'raw'/'Churn_Modelling_Germany.csv')

f1_score_list = send_sample_requests(df_germany)
f1_score_list


100%|██████████| 50/50 [00:52<00:00,  1.06s/it]


[0.411764705882353,
 0.3846153846153846,
 0.24999999999999997,
 0.4347826086956522,
 0.48275862068965514,
 0.43478260869565216,
 0.4799999999999999,
 0.5263157894736842,
 0.3448275862068965,
 0.37037037037037035,
 0.4545454545454545,
 0.6086956521739131,
 0.47058823529411764,
 0.5185185185185185,
 0.3478260869565218,
 0.4545454545454545,
 0.5,
 0.7272727272727274,
 0.37037037037037035,
 0.3478260869565218,
 0.33333333333333337,
 0.35714285714285715,
 0.5,
 0.5714285714285715,
 0.6842105263157894,
 0.5333333333333333,
 0.2,
 0.21052631578947367,
 0.25,
 0.2727272727272727,
 0.5263157894736842,
 0.43478260869565216,
 0.5806451612903226,
 0.4615384615384615,
 0.5,
 0.6206896551724138,
 0.4827586206896552,
 0.3571428571428571,
 0.4210526315789473,
 0.25,
 0.4166666666666667,
 0.7058823529411765,
 0.3478260869565218,
 0.2727272727272727,
 0.4864864864864865,
 0.48648648648648646,
 0.4615384615384615,
 0.5517241379310346,
 0.5833333333333334,
 0.43750000000000006]

### ... as expected, the values are much lower.
### But, typically, we can't compute model metrics on production data right away 
### because ground truth labels might not be available until much-much later


### All we can do is try and look at "proxy" metrics that measure statistical differences (distances) between train data and production data

In [29]:
x = requests.get(API_URL + '/drift_data')
if x.ok:
    data = x.json()
else:
    x.raise_for_status()

# df_p_vals = pd.DataFrame(data)
# df_p_vals

HTTPError: 500 Server Error: Internal Server Error for url: http://0.0.0.0:8080/drift_data

In [27]:
x

<Response [500]>

### Values below the threshold (e.g. 0.05) indicate data drift

In [34]:
import plotly_express as px

fig = px.line(df_p_vals, x='time', y=feat_cols)
fig.add_hline(y=0.05, line_color='red')